<a href="https://colab.research.google.com/github/hdakhli/mlops-2024/blob/main/30_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#
## Word Embeddings Avec 

In [ ]:
!pip install nltk
!pip install gensim

# Download EBOOK ALICE'S ADVENTURES IN WONDERLAND
# https://www.gutenberg.org/files/11/11-0.txt

In [ ]:
# Lire le fichier 'alice.txt'

In [ ]:
# Remplacer les retours à la ligne \n par des espaces

In [ ]:
# Extraire chaque phrase et les mots qui la compose en un tableau de tableau à l'aide des fonctions sent_tokenize et word_tokenize de nltk.tokenize

In [ ]:
# Créer 2 modèles: CBOW et Skip Gram à partir de ce tableau. cf doc: gensim.models.Word2Vec

In [ ]:
# Comparer les mots suivants avec les 2 modelès : 
# 'alice' vs 'wonderland'
# 'alice' vs 'machines'

#
#
## LLM




In [ ]:
# Download: https://huggingface.co/TheBloke/Llama-2-7b-Chat-GGUF/blob/main/llama-2-7b-chat.Q2_K.gguf

In [3]:
!pip install llama-cpp-python

  Using cached llama_cpp_python-0.2.76.tar.gz (49.4 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.76-cp311-cp311-macosx_14_0_x86_64.whl size=4105191 sha256=c50c5cfdfefcefc4388f42ec894d2a93e42e26a89c2f66b6ad005ee9afa29f43
  Stored in directory: /Users/j.sansa/Library/Caches/pip/wheels/78/ad/8f/d3fc4b4932eea66384e6a3ca2d500d82728a05d92be3c6246e
Successfully built llama-cpp-python


In [4]:
from llama_cpp import Llama

# Put the location of to the GGUF model that you've download from HuggingFace here
model_path = "llama-2-7b-chat.Q2_K.gguf"

# Create a model
model = Llama(model_path=model_path)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from llama-2-7b-chat.Q2_K.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32              = 32

In [6]:
# Prompt creation
system_message = "Vous êtes un assistant utile. Vous êtes là pour m'aider avec mes questions"
user_message = "Quelle est la capitale de la France ?"

prompt = f"""<s>[INST] <<SYS>>
{system_message}
<</SYS>>
{user_message} [/INST]"""

# Model parameters
max_tokens = 5

# Run the model
output = model(prompt, max_tokens=max_tokens, echo=True)

# Print the model output
print(output)

llama_tokenize_internal: Added a BOS token to the prompt as specified by the model but the prompt also starts with a BOS token. So now the final prompt starts with 2 BOS tokens. Are you sure this is what you want?


Llama.generate: prefix-match hit

llama_print_timings:        load time =   54982.81 ms
llama_print_timings:      sample time =       2.14 ms /     5 runs   (    0.43 ms per token,  2338.63 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (     nan ms per token,      nan tokens per second)
llama_print_timings:        eval time =    2477.79 ms /     5 runs   (  495.56 ms per token,     2.02 tokens per second)
llama_print_timings:       total time =    2487.05 ms /     5 tokens


{'id': 'cmpl-a71c9481-85a7-4608-ae2c-0ec19b8e3215', 'object': 'text_completion', 'created': 1716828375, 'model': 'llama-2-7b-chat.Q2_K.gguf', 'choices': [{'text': "<s>[INST] <<SYS>>\nVous êtes un assistant utile. Vous êtes là pour m'aider avec mes questions\n<</SYS>>\nQuelle est la capitale de la France ? [/INST]  Bonjour!", 'index': 0, 'logprobs': None, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 52, 'completion_tokens': 5, 'total_tokens': 57}}


In [ ]:
user_message = "Et celle de l'Espagne ? "
prompt = f"""<s>[INST] <<SYS>>
{system_message}
<</SYS>>
{user_message} [/INST]"""

output = model(prompt, max_tokens=max_tokens, echo=True)

print(output)

In [ ]:
# Que remarquez-vous ?
# Comment pourriez-vous résoudre le problème ?
# Exposer ce model en API REST avec FastAPI et Uvicorn

### LangChain

In [ ]:
!pip install langchain-core langchain-community

In [ ]:
from langchain_community.llms import LlamaCpp
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = LlamaCpp(
    model_path="llama-2-7b-chat.Q2_K.gguf", verbose=False
)

# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Vous êtes un assistant utile. Vous êtes là pour répondre à mes questions {input}.",
        )
    ]
)
output_parser = StrOutputParser()

chain = prompt | llm | output_parser

In [ ]:
chain.invoke({"Quelle est la capitale de la France ?"})

In [ ]:
chain.invoke({"Et celle de l'Espagne ?"})

In [ ]:
# Faites en sorte que le modèle prenne en compte l'historique de la conversation
# Exposer le model en API à l'aide de fast api et langserve